## Build a Simple LLM Application Using LCEL

##### Dependencies
* langchain
* langchain_groq
* langchain_core
* langserve
* fastapi
* uvicorn

In [2]:
import os
from dotenv import load_dotenv
load_dotenv()

GROQ_API_KEY=os.getenv("GROQ_API_KEY")

In [3]:
from langchain_groq import ChatGroq
model=ChatGroq(model="gemma2-9b-it",api_key=GROQ_API_KEY)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x00000215F12E0200>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x00000215F1021DC0>, model_name='gemma2-9b-it', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [4]:
from langchain_core.messages import SystemMessage, HumanMessage

messages=[
    SystemMessage(content="Translate the following from English to French"),
    HumanMessage(content="Hello")
]

In [5]:
result=model.invoke(messages)

In [6]:
result

AIMessage(content='Bonjour  \n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 5, 'prompt_tokens': 17, 'total_tokens': 22, 'completion_time': 0.009090909, 'prompt_time': 0.001913718, 'queue_time': 0.244999202, 'total_time': 0.011004627}, 'model_name': 'gemma2-9b-it', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--50316489-46eb-494c-89f7-37fa409bb86c-0', usage_metadata={'input_tokens': 17, 'output_tokens': 5, 'total_tokens': 22})

In [8]:
from langchain_core.output_parsers import StrOutputParser
parser=StrOutputParser()
response=parser.invoke(result)
response

'Bonjour  \n'

In [9]:
# Using LCEL - Chain components
chain=model|parser
chain.invoke(messages)

'Bonjour \n'

In [ ]:
## Using prompt template instaed of messages
from langchain_core.prompts import ChatPromptTemplate

generic_template="Translate the following into {language}"

prompt=ChatPromptTemplate.from_messages([
    ("system",generic_template),
    ("user","{text}")
])

In [11]:
result=prompt.invoke({"language":"French","text":"Hello"})
result

ChatPromptValue(messages=[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}), HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})])

In [13]:
result.to_messages()

[SystemMessage(content='Translate the following into French', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello', additional_kwargs={}, response_metadata={})]

In [14]:
## Changing together components with LCEL
chain=prompt|model|parser
chain.invoke({"language":"French","text":"Hello"})

'Bonjour \n'